In [1]:
links = [
    'https://chinapost.nownews.com/20210716-2654837',
    'https://chinapost.nownews.com/20210716-2654495',
    'https://chinapost.nownews.com/20210712-2648271',
    'https://chinapost.nownews.com/20210719-2658483',
    'https://chinapost.nownews.com/20210719-2654835'
]

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
# import nltk
import fasttext

In [3]:
def zng(paragraph):
    for sent in re.findall(u'[^!?。\.!\?」]+[!?。\.\!\?]?', paragraph, flags=re.U):
        yield sent

In [4]:
path_to_pretrained_model = './tmp/lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)

In [5]:
def scrap_and_clean_text(link):
    source = urlopen(link).read()
    soup = BeautifulSoup(source,'lxml')
    text = ''
    for paragraph in soup.find_all('p'):
        text += paragraph.text
        
    text = re.sub(r'【.*?】+', '', text)
    list_text = list(zng(text))
    
    for i in range(len(list_text)):
      if len(list_text[i]) <=3:
        list_text[i-1] += list_text[i]

    list_text = [x for x in list_text if not len(x) <= 3]
    
    for i in range(len(list_text)-1):
      if list_text[i][0] == ' ':
        if list_text[i+1][0] == ' ':
            list_text[i-1] += list_text[i] + list_text[i+1]
        else:
            list_text[i-1] += list_text[i]
        
    list_text[len(list_text)-2] += list_text[len(list_text)-1]

    list_text = [x for x in list_text if not x[0] == ' ']
    
    for _ in range(5): 
        for i in range(len(list_text)-1):
          lang1 = fmodel.predict(list_text[i])[0][0]
          lang2 = fmodel.predict(list_text[i+1])[0][0]
          if lang1 == lang2:
            list_text[i] += list_text[i+1]
            list_text[i+1] = ''
        list_text = [x for x in list_text if not x == '']
    
    
    en = []
    zh = []
    i=0

    while i <= len(list_text)-1:
        if i % 2 == 0:
            zh.append(list_text[i])
        else:
            en.append(list_text[i])
        i += 1
    
    return en, zh

In [6]:
en_all = []
zh_all = []

for link in links:
    en = []
    zh = []
    en, zh = scrap_and_clean_text(link)
    en_all.extend(en)
    zh_all.extend(zh)

In [7]:
len(en_all)

43

In [8]:
len(zh_all)

43

In [9]:
with open('english_ds.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in en_all:
        filehandle.write('%s\n' % listitem)

In [10]:
with open('chinese_ds.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in zh_all:
        filehandle.write('%s\n' % listitem)